In [1]:
import flatten_lattice as fl
import torch
from bert_models import LinearPOSBertV1
from encoding_utils import *
from transformers import AutoTokenizer
import pickle
from mask_utils import *
import json
import os
import numpy as np

device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
xlm_tok = fl.bert_tok


2022-09-25 06:36:30.374135: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-25 06:36:30.374157: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#V12 first attempt with inputs prepended
VNUM = 12
MOD_NAME = 'bertonewayv1.pth'

# specifies files for pre-loading
LOADED = {
    'amasks': 'attmasksallv'+str(VNUM)+'.pt',
    'tmaps': 'tmapsmaskedv'+str(VNUM)+'/'
}

In [39]:
def prepend_input(pgraph, inp):
    inptoks = xlm_tok(inp).input_ids
    posadd = len(inptoks)
    inpflat = []
    ind = 0
    for i in range(len(inptoks)):
        nl = []
        inp = inptoks[i]
        if i<(len(inptoks)-1):
            nl.append(str(inptoks[i+1])+" "+str(ind+1))
        inpflat.append({
            'token_idx':inp, 
            'pos':ind,
            'id': str(inp)+" "+str(ind),
            'nexts':nl,
            'score':0,
        })
        ind+=1
    inpflat[-1]['nexts'].append(pgraph[0]['id'].split()[0]+" "+str(posadd))
    inpflat.extend(pgraph)
    for i in range(posadd, len(inpflat)):
        extok = inpflat[i]
        extok['pos']+=posadd
        extok['id']= str(extok['token_idx'])+" "+str(extok['pos'])
        for j in range(len(extok['nexts'])):
            newpos = int(extok['nexts'][j].split()[1])+posadd
            extok['nexts'][j] = extok['nexts'][j].split()[0]+" "+str(newpos)
    return inpflat

In [40]:
STOPS = -1

# Get examples (just use the normal lattice examples ig?)
processedgraphs, inps, refs = fl.get_processed_graph_data(fl.frenbase, -1, STOPS)

# get exploded candidates to generate gold labels
resarrs = [fl.get_cover_paths(p)[0] for p in processedgraphs]

# extra step for greedy 
if STOPS==1:
    processedgraphs = filter_greedy(processedgraphs)
    

# ensure no empty examples
clean_empty(resarrs, processedgraphs)

processedgraphs=[prepend_input(processedgraphs[i], inps[i]) for i in range(len(processedgraphs))]


In [10]:
ctest = get_adjac_mat(processedgraphs[0])

In [42]:
processedgraphs[0][49]

{'token_idx': 581, 'pos': 49, 'id': '581 49', 'nexts': ['7082 50'], 'score': 0}

In [48]:
processedgraphs[0]

def get_validnext (pos, nlist):
    retval = ""
    for n in nlist:
        if pos< int(n.split()[1]):
            retval = n
            if "2 " not in n:
                return retval
    if len(retval)>0:
        return retval
    print("no valid")
    print(pos)
    print(nlist)
    return ""

def p_wnext(pgraph):
    nid = '0 0'
    for tokd in pgraph:
        if tokd['id']==nid:
            print(xlm_tok.decode(tokd['token_idx']))
            nid = get_validnext(tokd['pos'], tokd['nexts'])
            
            #print(tokd)
    print(nid)
            
p_wnext(processedgraphs[0])

<s>
Le
président
américain
devait
recevoir
vendredi
1
er
novembre
2013
le
premier
ministre
ir
a
kien
No
uri
Al
Malik
i
,
en
quê
te
d
'
aide
des
États
-
Unis
pour
lutte
r
contre
la
plus
forte
va
gue
de
violence
depuis
cinq
ans
.
</s>
The
US
President
was
to
receive
Iraq
i
Prime
Minister
No
uri
Al
Malik
i
Friday
,
November
1
,
2013
in
an
effort
to
se
ek
US
assistance
in
fighting
the
worst
wa
ve
of
violence
in
five
years
.
2 90


In [ ]:
# given a list of sub-scores (topological flattening of the graph), use dp to get the highest scoring path
# idlist has the corresponding graph ids for 
# would need to do a sort on pgrapaps that makes sure that no next node is before in the linear ordering
# reverse since we're using nexts
# TODO simplify code to not need so many data structures
def dp_best_path(pgraphs, graph):
    assert len(pgraphs)==len(scores)
    bplist = []
    bsco_list =[]
    for i in range(len(idlist)):
        bpath = []
        cur = pgraphs[i]
        # first instance, assume direct all endings to a single node, score 0
        if len(cur['nextlist'])==0:
            bpath.append(i)
            bplist.append(bpath)
            bsco_list.append(cur['score'])
            # check if this is how things work in python
            graph[pgraphs['id']]['bestsco'] = cur['score']
            graph[pgraphs['id']]['plist'] = bpath
            continue
        # get the highest prev from ahead to use
        mval = -1
        maxnext = None
        for n in cur['nextlist']:
            if graph[n]['bestsco']>mval:
                mval = graph[n]['bestsco']
                maxnext = graph[n]
        # add in scores / path from that prev
        bpath.extend(maxnext['plist']+[i])
        bplist.append(bpath)
        bsco_list.append(cur['score']+mval)
        graph[pgraphs['id']]['bestsco'] = cur['score']+mval
        graph[pgraphs['id']]['plist'] = bpath
    return bplist[-1], bsco_list[-1]

def get_idlist(pgraph):
    return [p['id'] for p in pgraph]
        
def dp_sort_pgraph(pgraphs):
    idlist = get_idlist(pgraph)
    for p in pgraphs:
        
    
        
        

In [32]:
# Attention mask code (TODO needs some updating)
if os.path.exists('./torchsaved/'+LOADED['amasks']):
        print("using loaded masks")
        attmasks = torch.load('./torchsaved/'+LOADED['amasks']).to(device)
else:
    print("creating new masks")
    masktmp = [connect_mat(p) for p in processedgraphs]
    attmasks = torch.stack(masktmp).to(device)
    torch.save(attmasks, './torchsaved/'+LOADED['amasks'])

creating new masks


In [49]:
ctest = connect_mat(processedgraphs[0])

In [50]:
posids[0]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
        87, 65, 66, 67, 68, 69, 70, 71, 72, 71, 69, 70, 71, 72, 73, 70, 71, 72,
        73, 74, 75, 76, 77, 78, 79, 80, 71, 72, 73, 74, 75, 76, 77, 78, 71, 67,
        68, 68, 69, 70, 70, 71, 68, 69, 70, 71, 72, 73, 71, 72, 72, 73, 74, 72,
        73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90,
        91, 92, 75, 76, 77, 78, 79, 80, 81, 82, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 82, 83, 85, 86, 83, 84, 87, 83, 78, 72, 73, 74, 75, 78, 75, 76, 77,
        75, 75, 76, 76, 77, 78, 80, 81, 

In [ ]:
# get tokenized inputs with posids (TODO needs an update for src/tgt format)
sents, posids = create_inputs(processedgraphs)

In [57]:
posids[0]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
        87, 65, 66, 67, 68, 69, 70, 71, 72, 71, 69, 70, 71, 72, 73, 70, 71, 72,
        73, 74, 75, 76, 77, 78, 79, 80, 71, 72, 73, 74, 75, 76, 77, 78, 71, 67,
        68, 68, 69, 70, 70, 71, 68, 69, 70, 71, 72, 73, 71, 72, 72, 73, 74, 72,
        73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90,
        91, 92, 75, 76, 77, 78, 79, 80, 81, 82, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 82, 83, 85, 86, 83, 84, 87, 83, 78, 72, 73, 74, 75, 78, 75, 76, 77,
        75, 75, 76, 76, 77, 78, 80, 81, 

In [56]:
# todo correct 101, 102 added before and after input
xlm_tok.decode(sents[0])

"_<s> Le président américain devait recevoir vendredi 1er novembre 2013 le premier ministre irakien Nouri Al Maliki, en quête d'aide des États-Unis pour lutter contre la plus forte vague de violence depuis cinq ans.</s> The US President was to receive Iraqi Prime Minister Nouri Al Maliki Friday, November 1, 2013 in an effort to seek US assistance in fighting the worst wave of violence in five years. 2013, in search of US assistance to fight the worst wave of violence in five years. on Friday 1 November 2013 in search of an 2013, seeking US assistance in in a bid to help the United States fight the worst an effort to seek US assistance in search, 1 November 2013 2013, seeking November 1 st 2013, in search, in an effort 2013 in an attempt to seek US assistance in fighting the worst wave of violence in five years. effort to find US support to fight the help the United States combat the worst wave the biggestve of strongest of worstek 2013, seeking US aid the support in fighting help assis

In [ ]:
cnt = 0
for t in processedgraphs[0]:
    if t['token_idx']==0:
        print(t)
cnt

{'token_idx': 0, 'pos': 0, 'id': '0 0', 'nexts': ['636 1'], 'score': 0}


0

In [13]:
bert_tok.decode(0)

'<s>'

In [10]:
processedgraphs[0][0]

{'token_idx': 0, 'pos': 0, 'id': '0 0', 'nexts': ['581 1'], 'score': 0}

In [67]:
bert_tok("<s>")

{'input_ids': [0, 0, 2], 'attention_mask': [1, 1, 1]}

In [ ]:
' </s> en_XX The US President was to receive Iraq i Prime Minister No uri Al Malik i Friday , November 1 , 2013 in an effort to se ek US assistance in fighting the worst wa ve of violence in five years'